In [12]:

%pip install pyspark
%pip install mysql-connector-python

import os
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import logging

# Configurer la journalisation
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Chemin local du fichier JAR MySQL Connector
mysql_connector_path = "./mysql-connector-java-8.0.25.jar"

# Configurer la session Spark avec le connecteur JDBC
spark = SparkSession.builder \
    .appName("MySQLConnection") \
    .config("spark.jars", mysql_connector_path) \
    .getOrCreate()

# Informations de connexion
jdbc_url = "jdbc:mysql://mysql-hackathonipssi.alwaysdata.net:3306/hackathonipssi_mia4"
connection_properties = {
    "user": "360556_root",
    "password": "hackathonipssi*",
    "driver": "com.mysql.cj.jdbc.Driver",
    "useSSL": "false",
    "requireSSL": "false",
    "verifyServerCertificate": "false"
}

logger.info("Chargement des données depuis MySQL...")

# Lire les données depuis les bases de données MySQL
medal_df = spark.read.jdbc(url=jdbc_url, table="Medal", properties=connection_properties)
host_df = spark.read.jdbc(url=jdbc_url, table="Host", properties=connection_properties)
athlete_df = spark.read.jdbc(url=jdbc_url, table="Athlete", properties=connection_properties)

logger.info("Jointure des tables...")

# Jointures nécessaires
df = medal_df.join(host_df, medal_df.slug_game == host_df.game_slug, "inner") \
             .join(athlete_df, medal_df.athlete_url == athlete_df.athlete_url, "inner")

df.show(5)

logger.info("Préparation des données...")

def prepare_data(df):
    # Encoder les colonnes string en valeurs numériques
    indexers = [
        StringIndexer(inputCol="event_gender", outputCol="event_gender_index"),
        StringIndexer(inputCol="country_code", outputCol="country_code_index"),
        StringIndexer(inputCol="medal_type", outputCol="medal_type_index"),
        StringIndexer(inputCol="game_location", outputCol="game_location_index")
    ]
    
    for indexer in indexers:
        df = indexer.fit(df).transform(df)
    
    # Sélection des caractéristiques et de la cible
    feature_cols = ["event_gender_index", "country_code_index", "game_year", "game_location_index", "athlete_year_birth"]
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    df = assembler.transform(df)
    
    return df

prepared_df = prepare_data(df)
prepared_df.show(5)

logger.info("Entraînement du modèle...")

def train_model(df):
    # Diviser les données en ensembles d'entraînement et de test
    train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
    
    # Définir les couches pour le réseau de neurones
    layers = [df.select("features").first()[0].size, 32, 16, 3]  # Entrée, couches cachées, sortie
    
    # Créer l'estimateur
    mlp = MultilayerPerceptronClassifier(layers=layers, featuresCol="features", labelCol="medal_type_index", maxIter=100)
    
    # Entraîner le modèle
    model = mlp.fit(train_df)
    
    # Évaluer le modèle
    result = model.transform(test_df)
    evaluator = MulticlassClassificationEvaluator(labelCol="medal_type_index", predictionCol="prediction", metricName="accuracy")
    accuracy = evaluator.evaluate(result)
    logger.info(f"Test set accuracy = {accuracy}")
    
    return model

model = train_model(prepared_df)

logger.info("Modèle entraîné avec succès.")


# Configurer la journalisation
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configurer la session Spark avec le connecteur JDBC
spark = SparkSession.builder \
    .appName("MySQLConnection") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.25") \
    .getOrCreate()

# Informations de connexion
jdbc_url = "jdbc:mysql://mysql-hackathonipssi.alwaysdata.net:3306/hackathonipssi_mia4"
connection_properties = {
    "user": "360556_root",
    "password": "hackathonipssi*",
    "driver": "com.mysql.cj.jdbc.Driver",
    "useSSL": "false",
    "requireSSL": "false",
    "verifyServerCertificate": "false"
}

logger.info("Chargement des données depuis MySQL...")

# Lire les données depuis les bases de données MySQL
medal_df = spark.read.jdbc(url=jdbc_url, table="Medal", properties=connection_properties)
host_df = spark.read.jdbc(url=jdbc_url, table="Host", properties=connection_properties)
athlete_df = spark.read.jdbc(url=jdbc_url, table="Athlete", properties=connection_properties)

logger.info("Jointure des tables...")

# Jointures nécessaires
df = medal_df.join(host_df, medal_df.slug_game == host_df.game_slug, "inner") \
             .join(athlete_df, medal_df.athlete_url == athlete_df.athlete_url, "inner")

df.show(5)

logger.info("Préparation des données...")

def prepare_data(df):
    # Encoder les colonnes string en valeurs numériques
    indexers = [
        StringIndexer(inputCol="event_gender", outputCol="event_gender_index"),
        StringIndexer(inputCol="country_code", outputCol="country_code_index"),
        StringIndexer(inputCol="medal_type", outputCol="medal_type_index"),
        StringIndexer(inputCol="game_location", outputCol="game_location_index")
    ]
    
    for indexer in indexers:
        df = indexer.fit(df).transform(df)
    
    # Sélection des caractéristiques et de la cible
    feature_cols = ["event_gender_index", "country_code_index", "game_year", "game_location_index", "athlete_year_birth"]
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    df = assembler.transform(df)
    
    return df

prepared_df = prepare_data(df)
prepared_df.show(5)

logger.info("Entraînement du modèle...")

def train_model(df):
    # Diviser les données en ensembles d'entraînement et de test
    train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
    
    # Définir les couches pour le réseau de neurones
    layers = [df.select("features").first()[0].size, 32, 16, 3]  # Entrée, couches cachées, sortie
    
    # Créer l'estimateur
    mlp = MultilayerPerceptronClassifier(layers=layers, featuresCol="features", labelCol="medal_type_index", maxIter=100)
    
    # Entraîner le modèle
    model = mlp.fit(train_df)
    
    # Évaluer le modèle
    result = model.transform(test_df)
    evaluator = MulticlassClassificationEvaluator(labelCol="medal_type_index", predictionCol="prediction", metricName="accuracy")
    accuracy = evaluator.evaluate(result)
    logger.info(f"Test set accuracy = {accuracy}")
    
    return model

model = train_model(prepared_df)

logger.info("Modèle entraîné avec succès.")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable